In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.feature_selection import SelectPercentile
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import ElasticNet
from sklearn.feature_selection import mutual_info_regression

from sklearn.svm import SVR
pd.set_option('display.max_rows', None,"display.max_columns", None)

Reading the Gene Expression w.r.t drug combinations


In [ ]:
gdscdata=pd.read_csv('/content/drive/MyDrive/FYP/gdsc_df.csv')

In [ ]:
gdscdata.shape

(122895, 1696)

In [ ]:
gdscdata.head(10)

Unnamed: 0 CELL_LINE_NAME  DRUG_NAME   LN_IC50      AAAS      AAMP  \
0           2          EW.11  Erlotinib  2.477990  0.564428  0.533984   
1           3        SK.ES.1  Erlotinib  2.033564  0.489227  0.275122   
2           4       COLO.829  Erlotinib  2.966007  0.203211  0.428943   
3           5           GB.1  Erlotinib  3.603243  0.365864  0.396098   
4           6       NCI.H720  Erlotinib  3.694213  0.397972  0.443252   
5           7      NCI.H1648  Erlotinib -3.130315  0.309252  0.652033   
6           8      NCI.H1650  Erlotinib  3.661843  0.399662  0.516748   
7           9      NCI.H1770  Erlotinib  3.259282  0.517110  0.339837   
8          10      NCI.H1838  Erlotinib  2.322838  0.390790  0.441626   
9          11        DMS.114  Erlotinib  3.920006  0.623997  0.591220   

      AARS2  AASDHPPT      AATF     ABCB7     ABCE1     ABCF1    ABHD11  \
0  0.548709  0.708049  0.631070  0.527553  0.628643  0.614949  0.275544   
1  0.450509  0.601296  0.207059  0.663132  0.611457  0.416082  0.279732   
2  0.677817  0.593281  0.529451  0.541277  0.539477  0.346465  0.527359   
3  0.293623  0.585437  0.715827  0.541069  0.488169  0.509984  0.348269   
4  0.619131  0.573670  0.602293  0.557288  0.468742  0.531031  0.725293   
5  0.471244  0.476978  0.586106  0.406114  0.184558  0.343767  0.613763   
6  0.339593  0.605048  0.507194  0.356831  0.491407  0.397733  0.568118   
7  0.411385  0.743008  0.629272  0.739239  0.616936  0.413114  0.397125   
8  0.539124  0.680764  0.702563  0.519235  0.723786  0.482191  0.535734   
9  0.640258  0.622954  0.587005  0.724267  0.684433  0.639234  0.327889   

       ABL1      ABT1     ACBD4     ACIN1      ACLY      ACO2     ACTG2  \
0  0.725189  0.482830  0.118721  0.694910  0.834281  0.400000  0.499042   
1  0.838643  0.468262  0.269635  0.607925  0.473565  0.408588  0.384525   
2  0.498281  0.476587  0.230365  0.491946  0.675668  0.306667  0.040762   
3  0.801742  0.422477  0.531050  0.375322  0.572200  0.455819  0.442529   
4  0.616319  0.624610  0.198858  0.535116  0.603184  0.273220  0.060026   
5  0.327298  0.345994  0.846575  0.501289  0.418704  0.485650  0.065028   
6  0.580564  0.431322  0.180365  0.281894  0.521035  0.598870  0.051405   
7  0.584002  0.377992  0.055936  0.770296  0.829164  0.536497  0.064070   
8  0.409581  0.576223  0.586073  0.834407  0.583002  0.456497  0.063006   
9  0.512491  0.682102  0.218037  0.564755  0.754406  0.377175  0.037250   

     ACTL6A    ACTR10     ACTR2     ACTR8     ADRM1      ADSL    AFG3L2  \
0  0.542811  0.288365  0.773415  0.621649  0.626695  0.561644  0.655836   
1  0.416146  0.466148  0.551336  0.719173  0.167520  0.642285  0.615686   
2  0.544170  0.462889  0.591889  0.579780  0.527870  0.712070  0.704015   
3  0.393313  0.494233  0.612810  0.556548  0.399518  0.577152  0.572176   
4  0.602338  0.460883  0.544255  0.347970  0.363362  0.326441  0.709430   
5  0.505300  0.553410  0.464757  0.581568  0.454052  0.469113  0.604669   
6  0.340038  0.439819  0.652720  0.597907  0.462489  0.423365  0.594585   
7  0.443055  0.509779  0.448021  0.480725  0.507683  0.632980  0.818487   
8  0.511280  0.617603  0.516897  0.451111  0.573667  0.601189  0.695612   
9  0.595814  0.494734  0.661410  0.778912  0.560410  0.603774  0.704949   

      AGAP5      AGO2    AGPAT1    AHCTF1      AHCY    AHCYL1     AIFM3  \
0  0.257329  0.524360  0.643890  0.787635  0.698627  0.504975  0.043910   
1  0.310335  0.437610  0.536575  0.778606  0.642809  0.540570  0.053050   
2  0.239858  0.632317  0.540977  0.637684  0.663741  0.307981  0.140672   
3  0.551969  0.483526  0.671977  0.656133  0.527571  0.954897  0.069144   
4  0.544566  0.536278  0.686020  0.718548  0.692100  0.450144  0.048679   
5  0.199882  0.694883  0.395095  0.765260  0.819716  0.698430  0.195112   
6  0.266213  0.508237  0.392161  0.676742  0.529597  0.452797  0.204053   
7  0.588984  0.537855  0.439950  0.885182  0.681747  0.644926  0.073912   
8  0.307077  0.446547  0.307273  0.690285  0.7

In [ ]:
gdscdata.drop(columns=['Unnamed: 0','CELL_LINE_NAME','DRUG_NAME'],inplace=True)

In [ ]:
X=gdscdata.drop(['LN_IC50'],axis=1)
Y=gdscdata['LN_IC50']

In [ ]:
gdscdata=[]

In [ ]:
X.head(10)

AAAS      AAMP     AARS2  AASDHPPT      AATF     ABCB7     ABCE1  \
0  0.564428  0.533984  0.548709  0.708049  0.631070  0.527553  0.628643   
1  0.489227  0.275122  0.450509  0.601296  0.207059  0.663132  0.611457   
2  0.203211  0.428943  0.677817  0.593281  0.529451  0.541277  0.539477   
3  0.365864  0.396098  0.293623  0.585437  0.715827  0.541069  0.488169   
4  0.397972  0.443252  0.619131  0.573670  0.602293  0.557288  0.468742   
5  0.309252  0.652033  0.471244  0.476978  0.586106  0.406114  0.184558   
6  0.399662  0.516748  0.339593  0.605048  0.507194  0.356831  0.491407   
7  0.517110  0.339837  0.411385  0.743008  0.629272  0.739239  0.616936   
8  0.390790  0.441626  0.539124  0.680764  0.702563  0.519235  0.723786   
9  0.623997  0.591220  0.640258  0.622954  0.587005  0.724267  0.684433   

      ABCF1    ABHD11      ABL1      ABT1     ACBD4     ACIN1      ACLY  \
0  0.614949  0.275544  0.725189  0.482830  0.118721  0.694910  0.834281   
1  0.416082  0.279732  0.838643  0.468262  0.269635  0.607925  0.473565   
2  0.346465  0.527359  0.498281  0.476587  0.230365  0.491946  0.675668   
3  0.509984  0.348269  0.801742  0.422477  0.531050  0.375322  0.572200   
4  0.531031  0.725293  0.616319  0.624610  0.198858  0.535116  0.603184   
5  0.343767  0.613763  0.327298  0.345994  0.846575  0.501289  0.418704   
6  0.397733  0.568118  0.580564  0.431322  0.180365  0.281894  0.521035   
7  0.413114  0.397125  0.584002  0.377992  0.055936  0.770296  0.829164   
8  0.482191  0.535734  0.409581  0.576223  0.586073  0.834407  0.583002   
9  0.639234  0.327889  0.512491  0.682102  0.218037  0.564755  0.754406   

       ACO2     ACTG2    ACTL6A    ACTR10     ACTR2     ACTR8     ADRM1  \
0  0.400000  0.499042  0.542811  0.288365  0.773415  0.621649  0.626695   
1  0.408588  0.384525  0.416146  0.466148  0.551336  0.719173  0.167520   
2  0.306667  0.040762  0.544170  0.462889  0.591889  0.579780  0.527870   
3  0.455819  0.442529  0.393313  0.494233  0.612810  0.556548  0.399518   
4  0.273220  0.060026  0.602338  0.460883  0.544255  0.347970  0.363362   
5  0.485650  0.065028  0.505300  0.553410  0.464757  0.581568  0.454052   
6  0.598870  0.051405  0.340038  0.439819  0.652720  0.597907  0.462489   
7  0.536497  0.064070  0.443055  0.509779  0.448021  0.480725  0.507683   
8  0.456497  0.063006  0.511280  0.617603  0.516897  0.451111  0.573667   
9  0.377175  0.037250  0.595814  0.494734  0.661410  0.778912  0.560410   

       ADSL    AFG3L2     AGAP5      AGO2    AGPAT1    AHCTF1      AHCY  \
0  0.561644  0.655836  0.257329  0.524360  0.643890  0.787635  0.698627   
1  0.642285  0.615686  0.310335  0.437610  0.536575  0.778606  0.642809   
2  0.712070  0.704015  0.239858  0.632317  0.540977  0.637684  0.663741   
3  0.577152  0.572176  0.551969  0.483526  0.671977  0.656133  0.527571   
4  0.326441  0.709430  0.544566  0.536278  0.686020  0.718548  0.692100   
5  0.469113  0.604669  0.199882  0.694883  0.395095  0.765260  0.819716   
6  0.423365  0.594585  0.266213  0.508237  0.392161  0.676742  0.529597   
7  0.632980  0.818487  0.588984  0.537855  0.439950  0.885182  0.681747   
8  0.601189  0.695612  0.307077  0.446547  0.307273  0.690285  0.704704   
9  0.603774  0.704949  0.299378  0.551700  0.857263  0.758979  0.530272   

     AHCYL1     AIFM3       AK2       AK6   AKIRIN2  ALDH18A1     ALDOA  \
0  0.504975  0.043910  0.538681  0.556965  0.526316  0.657201  0.589820   
1  0.540570  0.053050  0.581870  0.642827  0.510297  0.718544  0.285520   
2  0.307981  0.140672  0.636687  0.495010  0.425629  0.692658  0.695427   
3  0.954897  0.069144  0.518272  0.614969  0.493447  0.626765  0.461078   
4  0.450144  0.048679  0.196251  0.665696  0.427918  0.727957  0.590909   
5  0.698430  0.195112  0.603465  0.451559  0.528396  0.675714  0.475776   
6  0.452797  0.204053  0.585904  0.440956  0.522571  0.600251  0.664398   
7  0.644926  0.073912  0.733745  0.595842  0.469316  0.616410  0.636908   
8  0.433783  0.108087  0.648

In [ ]:
X,x_test,Y,y_test = train_test_split(X,Y)


In [ ]:

selected_top_columns = SelectPercentile(mutual_info_regression, percentile=50)
selected_top_columns.fit(X, Y)
selected_top_columns.get_support()


array([False,  True, False, ..., False, False,  True])

In [ ]:
X.columns[selected_top_columns.get_support()]


Index(['AAMP', 'ABCB7', 'ABCE1', 'ABHD11', 'ABL1', 'ABT1', 'ACIN1', 'ACLY',
       'ACTL6A', 'ACTR10',
       ...
       'ZC3H4', 'ZMAT2', 'ZMAT5', 'ZNF131', 'ZNF219', 'ZNF22', 'ZNF407',
       'ZNF622', 'ZNF830', 'DRUG_ID'],
      dtype='object', length=846)

In [ ]:
li=list(X.columns[selected_top_columns.get_support()]
)

In [ ]:
with open('collistinfogainnew.txt', 'w+') as f:
     
    # write elements of list
    for items in li:
        f.write('%s\n' %items)
     
    print("File written successfully")
 
 
# close the file
f.close()

File written successfully


In [ ]:
X=X[X.columns[selected_top_columns.get_support()]]
x_train=x_train[X.columns[selected_top_columns.get_support()]]

In [ ]:
X.shape

(92171, 846)

In [ ]:
!pip install zoofs


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 139 kB 10.3 MB/s 
     |████████████████████████████████| 132 kB 46.5 MB/s 


In [ ]:
from sklearn.metrics import mean_squared_error
def objective_function_topass(model,X_train, y_train, X_valid, y_valid):      
    model.fit(X_train,y_train)  
    P=mean_squared_error(y_valid,model.predict_proba(X_valid))
    return P
    

from zoofs import ParticleSwarmOptimization
algo_object=ParticleSwarmOptimization(objective_function_topass,n_iteration=100,
                                       population_size=20,minimize=True)
from sklearn.linear_model import LinearRegression()
lgb_model = LinearRegression()                            
algo_object.fit(lgb_model,X_train, y_train,X_train, y_train, verbose=True)


Selecting features with genetic algorithm.


In [ ]:
algo_object.plot_history()

In [ ]:
li=list(algo_object.best_feature_list
)

In [ ]:
with open('collistinfogainnew.txt', 'w+') as f:
     
    # write elements of list
    for items in li:
        f.write('%s\n' %items)
     
    print("File written successfully")
 
 
# close the file
f.close()

File written successfully
